In [1]:
import numpy as np
import scipy.signal

In [17]:
def cond_lag_lim_vec_fast(cond_chunk,tstep,samp_freq,lag_len,vol,T):
    
    #conversion from LAMMPS real units to SI units
    
    e2coul = 1.6021*10**-19
    ang2m = 1*10**-10
    fs2s = 1*10**-15
    curr_conv = (e2coul*ang2m)/fs2s
    kB = 1.3806504e-23                         # in J/K
    conv = ((curr_conv**2)*fs2s)/(ang2m**3)
    
    c = []
    inte = []
    
    #sampling of charge current every lag_len timeteps
    
    shift = lag_len/samp_freq
    a = np.arange(0,cond_chunk.shape[0],shift)
    cond_x = np.array([cond_chunk[int(i)][0] for i in a])
    cond_y = np.array([cond_chunk[int(i)][1] for i in a])
    cond_z = np.array([cond_chunk[int(i)][2] for i in a])
    
    #main code to calculate autocorrelation of charge current followed by ensemble averaging 
    
    acf_temp_x = scipy.signal.correlate(cond_x,cond_x)
    acf_temp_y = scipy.signal.correlate(cond_y,cond_y)
    acf_temp_z = scipy.signal.correlate(cond_z,cond_z)
    
    acf_x = np.flip(acf_temp_x[0:len(cond_x)])
    acf_y = np.flip(acf_temp_y[0:len(cond_y)])
    acf_z = np.flip(acf_temp_z[0:len(cond_z)])
    acf_tot = acf_x + acf_y + acf_z
    
    norm = np.flip(np.arange(1,len(cond_x)+1,1))
    acf_norm = acf_tot/norm
    
    #integration of autocorrelation function to calculate conductivity in SI units
    
    for i in range(len(acf_tot)):
        temp = np.trapz(acf_norm[0:i+1],dx=(lag_len*tstep))
        temp_c = (conv*temp)/(3*kB*T*vol)
        inte.append(temp)
        c.append(temp_c)
       
    return acf_norm,inte,c

In [18]:
filename = '/Users/rajorshi/Desktop/CMD/Conductivity/Reaxff/673K_5GPa_new/Cond_lim/extract.txt'       #path of input data file containing charge current data
filename_prog = '/Users/rajorshi/Desktop/CMD/Conductivity/Reaxff/673K_5GPa_new/Cond_lim/output.out'   #path of ouput file

f_prog = open(filename_prog,'w')

#load data into numpy arrays

cond = np.loadtxt(filename,skiprows=2,usecols=(1,2,3))
t = np.loadtxt(filename,skiprows=2,usecols=(0))
    
samp_freq = 2            #sampling frequency in timesteps
lag_len = 2              #difference between successive lag times (in timesteps) where autocorrelation is calculated. Has to be multiple of sampling frequency
time_chunk = 2           #input data is broken down into chunks of size time_chunk (in picosecond). Each chunk can then be used for analysis
tstep = 0.5              #timestep of MD simulations in femtoseconds

#simulation parameters

box_max = 28.496098      
box_min = 3.5039283
vol = (box_max-box_min)**3
T = 673

#extract charge current data from input file in chunks of size time_chunk and store them in dictionary

steps = time_chunk*1000*(1/tstep)
steps_arr = steps/samp_freq

n_chunk = 2           #number of time chunks of size time_chunk for conductivity calculation. Important for assessing convergence wrt simulation length
cond_chunk = {}
start = int(len(cond)-steps_arr)
end = int(len(cond))

#number of timesteps in input.dat file must be atleast time_chunk*t_step*n_chunk
while start>=0:
    temp = cond[start-1:end][:]
    cond_chunk.update({t[start-1]:temp})
    end = int(start)
    start = int(end - steps_arr)
        
count = 0

#write out important simulation parameters in output file

f_prog.write('Box size: max: '+str(box_max)+' min: '+str(box_min)+' in Ang'+'\n')
f_prog.write('Timestep: '+str(tstep)+'\n')
f_prog.write('T: '+str(T)+'\n')

#iterate over dictionary entries and calculate conductivity for n_chunk time chunks

for key in cond_chunk.keys():
    lag = 0
    count = count + 1
    
    if count <= n_chunk:
        print('Calculating for chunck {0}'.format(count))
        
        acf_norm,inte,c = cond_lag_lim_vec_fast(cond_chunk[key],tstep,samp_freq,lag_len,vol,T)
        f_prog.write('Chunk '+str(count)+' of size '+str(time_chunk)+'ps'+'\n')
        f_prog.write('Start time: '+str((int(key)*tstep)/1000)+'ps'+'\n')
        f_prog.write('Lag'+'    '+'Lag (in ps)'+'     '+'Auto-correlation'+'     '+'Integral(acf)'+'        '+'Conductivity'+'\n')
        for i in range(len(acf_norm)):
            f_prog.write(str(lag)+'      '+str((lag*tstep)/1000)+'          '+str(acf_norm[i])+'    '+str(inte[i])+'    '+str(c[i])+'\n') 
            lag = lag + lag_len
        
f_prog.close()

Calculating for chunck 1
Calculating for chunck 2
